In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files
import nltk
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df=pd.read_csv("/content/Input.xlsx - Sheet1.csv")[['URL_ID','URL']]
df.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [5]:
df.drop('URL_ID',axis=1,inplace=True)

In [6]:
df.shape

(100, 1)

In [7]:
df

,URL
0,https://insights.blackcoffer.com/rising-it-cit...
1,https://insights.blackcoffer.com/rising-it-cit...
2,https://insights.blackcoffer.com/internet-dema...
3,https://insights.blackcoffer.com/rise-of-cyber...
4,https://insights.blackcoffer.com/ott-platform-...
...,...
95,https://insights.blackcoffer.com/what-is-the-r...
96,https://insights.blackcoffer.com/impact-of-cov...
97,https://insights.blackcoffer.com/contribution-...
98,https://insights.blackcoffer.com/how-covid-19-...


In [23]:
%%time
def extract_article_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for element in soup(["header", "footer"]):
            element.decompose()
        article_title = soup.find('title').text.strip()
        article_text = ""
        article_div = soup.find('div', class_='td-post-content tagdiv-type')
        if article_div:
            article_text = article_div.get_text()
        return article_title, article_text

    except Exception:
        print(f"Error while extracting article from {url}: {Exception}")
        return None, None

def save_article_to_file(url_id, article_title, article_text):
    if not os.path.exists("articles"):
        os.mkdir("articles")

    with open(f"articles/{url_id}.txt", "w", encoding="utf-8") as file:
        file.write(f"Title: {article_title}\n\n")
        file.write(article_text)

def main():
    input_file = "/content/Input (1).xlsx"
    df = pd.read_excel(input_file)

    for index, row in df.iterrows():
        url_id = row["URL_ID"]
        url = row["URL"]
        article_title, article_text = extract_article_text(url)
        if article_title and article_text:
            save_article_to_file(url_id, article_title, article_text)
            print(f"Article {url_id} extracted and saved successfully.")
        else:
            print(f"Failed to extract article {url_id}.")

if __name__ == "__main__":
    main()

Article blackassign0001 extracted and saved successfully.
Article blackassign0002 extracted and saved successfully.
Article blackassign0003 extracted and saved successfully.
Article blackassign0004 extracted and saved successfully.
Article blackassign0005 extracted and saved successfully.
Article blackassign0006 extracted and saved successfully.
Article blackassign0007 extracted and saved successfully.
Article blackassign0008 extracted and saved successfully.
Article blackassign0009 extracted and saved successfully.
Article blackassign0010 extracted and saved successfully.
Article blackassign0011 extracted and saved successfully.
Article blackassign0012 extracted and saved successfully.
Article blackassign0013 extracted and saved successfully.
Failed to extract article blackassign0014.
Article blackassign0015 extracted and saved successfully.
Article blackassign0016 extracted and saved successfully.
Article blackassign0017 extracted and saved successfully.
Article blackassign0018 extra

In [29]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')
import re
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
def load_dictionaries(positive_dict_file, negative_dict_file):
    with open(positive_dict_file, 'r', errors='ignore') as file:
        positive_words = set(file.read().splitlines())
    with open(negative_dict_file, 'r', errors='ignore') as file:
        negative_words = set(file.read().splitlines())
    return positive_words, negative_words

# Function to perform sentiment analysis and calculate scores
def calculate_sentiment_scores(text, positive_words, negative_words):
    sia = SentimentIntensityAnalyzer()
    tokens = word_tokenize(text)

    positive_score = 0
    negative_score = 0

    for word in tokens:
        # Remove punctuation and convert to lowercase
        word = word.lower()
        if word.isalpha():
            # Check if the word is in the positive dictionary
            if word in positive_words:
                positive_score += 1
            # Check if the word is in the negative dictionary
            if word in negative_words:
                negative_score += 1
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)

    return positive_score, negative_score, polarity_score, subjectivity_score

def main():
    input_data_file = "/content/Output Data Structure.xlsx"
    positive_dict_file = "/content/positive-words.txt"
    negative_dict_file = "/content/negative-words.txt"
    articles_dir = "articles"

    positive_words, negative_words = load_dictionaries(positive_dict_file, negative_dict_file)
    output_data = pd.read_excel(input_data_file)

    results = []
    for index, row in output_data.iterrows():
        url_id = row["URL_ID"]
        url = row["URL"]
        article_file = os.path.join(articles_dir, f"{url_id}.txt")

        if os.path.exists(article_file):
          with open(article_file, 'r', encoding='utf-8') as article:
            article_text = article.read()
            positive_score, negative_score, polarity_score, subjectivity_score = calculate_sentiment_scores(article_text, positive_words, negative_words)

            results.append({
                "URL_ID": url_id,
                "URL": url,
                "Positive_Score": positive_score,
                "Negative_Score": negative_score,
                "Polarity_Score": polarity_score,
                "Subjectivity_Score": subjectivity_score
            })
    result_df = pd.DataFrame(results)
    result_df.to_excel("output.xlsx", index=False)

if __name__ == "__main__":
    main()

In [27]:
sentiment_score = pd.read_excel("output.xlsx")

In [28]:
sentiment_score

,URL_ID,URL,Positive_Score,Negative_Score,Polarity_Score,Subjectivity_Score
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,44,6,0.760000,0.035765
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,64,31,0.347368,0.055458
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,40,24,0.250000,0.051780
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,39,75,-0.315789,0.092457
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,24,8,0.500000,0.041078
...,...,...,...,...,...,...
84,blackassign0094,https://insights.blackcoffer.com/gaming-disord...,34,50,-0.190476,0.064319
85,blackassign0095,https://insights.blackcoffer.com/what-is-the-r...,13,25,-0.315789,0.051007
86,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,30,57,-0.310345,0.070789
87,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,32,35,-0.044776,0.054828


###  Defining the functions to calculate the tasks


In [30]:
#for avearage sentence length
def calculate_avg_sentence_length(sentences):
    total_words = sum(len(word_tokenize(sentence)) for sentence in sentences)
    total_sentences = len(sentences)
    return total_words / total_sentences

In [31]:
#calculate percentage of complex words
def calculate_percentage_complex_words(text):
    words = word_tokenize(text)
    complex_words = [word for word in words if len(word) > 2]
    return len(complex_words) / len(words)

In [32]:
#calculate fog index
def calculate_fog_index(avg_sentence_length, percentage_complex_words):
    return 0.4 * (avg_sentence_length + percentage_complex_words)


In [33]:
#to calculate average number of words per sentence
def calculate_avg_words_per_sentence(words, sentences):
    return len(words) / len(sentences)

In [34]:
#calculate complex word count
def calculate_complex_word_count(text):
    words = word_tokenize(text)
    complex_words = [word for word in words if len(word) > 2]
    return len(complex_words)

In [35]:
#calculate word count
def calculate_word_count(text):
    words = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    cleaned_words = [word for word in words if word not in stop_words and word.isalpha()]
    return len(cleaned_words)

In [36]:
#count syllables in a word
def count_syllables(word):
    vowels = "aeiouAEIOU"
    count = 0
    if word[-1] in ['e', 'E'] and word[-2:] != 'le' and word[-2:] != 'LE':
        word = word[:-1]
    for index, letter in enumerate(word):
        if index == 0 and letter in vowels:
            count += 1
        elif letter in vowels and word[index-1] not in vowels:
            count += 1
    return count

In [37]:
#to calculate syllable count per word
def calculate_syllable_count_per_word(text):
    words = word_tokenize(text)
    syllable_count = sum(count_syllables(word) for word in words)
    return syllable_count / len(words)

In [38]:
#to calculate personal pronoun count
def calculate_personal_pronouns(text):
    pronouns = ["I", "we", "my", "ours", "us"]
    pattern = r'\b(?:' + '|'.join(pronouns) + r')\b'
    matches = re.findall(pattern, text)
    return len(matches)

In [39]:
#to calculate average word length
def calculate_avg_word_length(text):
    words = word_tokenize(text)
    total_characters = sum(len(word) for word in words)
    return total_characters / len(words)

###  The above functions are defined as per the given instructions.

###  Defining Main Function

In [40]:
def main():
    output_data_file = "/content/Output Data Structure.xlsx"
    articles_dir = "articles"

    # Read output data structure Excel file
    output_data = pd.read_excel(output_data_file)

    results_ = []
    for index, row in output_data.iterrows():
        url_id = row["URL_ID"]
        article_file = os.path.join(articles_dir, f"{url_id}.txt")

        if os.path.exists(article_file):
            with open(article_file, 'r', encoding='utf-8') as article:
                article_text = article.read()
            sentences = sent_tokenize(article_text)
            words = word_tokenize(article_text)
            avg_sentence_length = calculate_avg_sentence_length(sentences)
            percentage_complex_words = calculate_percentage_complex_words(article_text)
            fog_index = calculate_fog_index(avg_sentence_length, percentage_complex_words)
            avg_words_per_sentence = calculate_avg_words_per_sentence(words, sentences)
            complex_word_count = calculate_complex_word_count(article_text)
            word_count = calculate_word_count(article_text)
            syllable_count_per_word = calculate_syllable_count_per_word(article_text)
            personal_pronoun_count = calculate_personal_pronouns(article_text)
            avg_word_length = calculate_avg_word_length(article_text)

            results_.append({
                "URL_ID": url_id,
                "Avg_Sentence_Length": avg_sentence_length,
                "Percentage_Complex_Words": percentage_complex_words,
                "Fog_Index": fog_index,
                "Avg_Words_Per_Sentence": avg_words_per_sentence,
                "Complex_Word_Count": complex_word_count,
                "Word_Count": word_count,
                "Syllable_Count_Per_Word": syllable_count_per_word,
                "Personal_Pronoun_Count": personal_pronoun_count,
                "Avg_Word_Length": avg_word_length
            })
    result_df2 = pd.DataFrame(results_)

    # Save results to Excel
    result_df2.to_excel("analysis.xlsx", index=False)

if __name__ == "__main__":
    main()


In [41]:
sentiment_analysis = pd.read_excel("analysis.xlsx")
sentiment_analysis

,URL_ID,Avg_Sentence_Length,Percentage_Complex_Words,Fog_Index,Avg_Words_Per_Sentence,Complex_Word_Count,Word_Count,Syllable_Count_Per_Word,Personal_Pronoun_Count,Avg_Word_Length
0,blackassign0001,17.696203,0.671674,7.347151,17.696203,939,733,1.246781,6,4.195994
1,blackassign0002,21.412500,0.717455,8.851982,21.412500,1229,896,1.499708,3,4.872154
2,blackassign0003,21.684211,0.735437,8.967859,21.684211,909,696,1.702265,13,5.492718
3,blackassign0004,23.711538,0.716139,9.771071,23.711538,883,690,1.593674,4,5.298459
4,blackassign0005,19.475000,0.752246,8.090899,19.475000,586,426,1.494223,6,5.077022
...,...,...,...,...,...,...,...,...,...,...
84,blackassign0094,20.406250,0.702910,8.443664,20.406250,918,666,1.315467,11,4.376723
85,blackassign0095,20.694444,0.693960,8.555362,20.694444,517,374,1.354362,4,4.495302
86,blackassign0096,24.580000,0.739626,10.127850,24.580000,909,657,1.485761,2,4.943857
87,blackassign0097,31.333333,0.711948,12.818112,31.333333,870,539,1.276596,5,4.327332


In [43]:
combined_df = pd.merge(sentiment_score, sentiment_analysis, on='URL_ID')
combined_df

,URL_ID,URL,Positive_Score,Negative_Score,Polarity_Score,Subjectivity_Score,Avg_Sentence_Length,Percentage_Complex_Words,Fog_Index,Avg_Words_Per_Sentence,Complex_Word_Count,Word_Count,Syllable_Count_Per_Word,Personal_Pronoun_Count,Avg_Word_Length
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,44,6,0.760000,0.035765,17.696203,0.671674,7.347151,17.696203,939,733,1.246781,6,4.195994
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,64,31,0.347368,0.055458,21.412500,0.717455,8.851982,21.412500,1229,896,1.499708,3,4.872154
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,40,24,0.250000,0.051780,21.684211,0.735437,8.967859,21.684211,909,696,1.702265,13,5.492718
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,39,75,-0.315789,0.092457,23.711538,0.716139,9.771071,23.711538,883,690,1.593674,4,5.298459
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,24,8,0.500000,0.041078,19.475000,0.752246,8.090899,19.475000,586,426,1.494223,6,5.077022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,blackassign0094,https://insights.blackcoffer.com/gaming-disord...,34,50,-0.190476,0.064319,20.406250,0.702910,8.443664,20.406250,918,666,1.315467,11,4.376723
85,blackassign0095,https://insights.blackcoffer.com/what-is-the-r...,13,25,-0.315789,0.051007,20.694444,0.693960,8.555362,20.694444,517,374,1.354362,4,4.495302
86,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,30,57,-0.310345,0.070789,24.580000,0.739626,10.127850,24.580000,909,657,1.485761,2,4.943857
87,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,32,35,-0.044776,0.054828,31.333333,0.711948,12.818112,31.333333,870,539,1.276596,5,4.327332


In [45]:
combined_df.to_excel("OutputDataStructure.xlsx")

In [46]:
from google.colab import files
files.download('OutputDataStructure.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>